In [ ]:
# Import the necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Set the desired figure size in inche
plt.rcParams['figure.figsize'] = (20, 12)  # Width: 8 inches, Height: 6 inches

#create my color array (viridis)
my_colors = ["#fde725",  "#90d743", "#35b779", "#21918c", "#31688e", "#443983", "#440154"]


# add color array to set_palette
sns.set_theme(font_scale=1.6, style="ticks", palette= my_colors)


In [ ]:
# My file path
myPath = '..\\data\\AGE\\'

# This window is used to calculate the moving average of the data
window = 30

In [ ]:
def readWaterAge(FolderName):
    # Read the data from the csv file and store it in a variable
    myVar = pd.read_csv(FolderName, sep = ",", header = 0, decimal = ".", index_col = 'date and time', parse_dates=['date and time'])
    myVar.rename(columns = {'calculated  AGE  for water from source 1 (d)': 'WaterAge'}, inplace = True)
    myVar.index.names = ['Date']
    myVar.dropna()
    myVar = myVar.iloc[4284:]
    myVar['WaterAge'] = myVar['WaterAge'].astype(float)
    return myVar

In [ ]:
def smoothMyVar(var, myWindow):    
   # Calculate the rolling mean and standard deviation for the variable
    roll = var.WaterAge.rolling(myWindow, center=True)
    mySmoothValues = roll.agg(['mean', 'std']) 
    return mySmoothValues

In [ ]:
def plotSmootWaterAge(var, myWindow, myLabel):
    # Plot the rolling mean and standard deviation of the water age
    smoothValues = smoothMyVar(var, myWindow)
        
    ax = smoothValues['mean'].plot(linewidth=3, label= myLabel)            
    ax.fill_between(smoothValues.index, smoothValues['mean'] - smoothValues['std'], 
                    smoothValues['mean'] + smoothValues['std'], alpha=0.2)
          
    plt.tight_layout()
    plt.ylabel("Age trazer (days)", size = 28)
    plt.xlabel('Time (years)', size = 28)

    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)

    plt.legend() 
          
    return ax

In [ ]:
def date2deltaTime(var):
    # Calculate the time in years from using  the first data point
    ser_diff = var.index.to_series().diff()
    var['deltaT'] = ser_diff.dt.seconds.div(60, fill_value=0)
    var['deltaT'] = ((var['deltaT'].cumsum()))/(60*24*365)
    var.set_index('deltaT', inplace=True)
    return var   

In [ ]:
def plotTerminosAgesZones(path, myWindow):
    # Read each area of the lagoon and plot the water age time series
    PRealFile = path + '01_PReal_inlet.csv'
    
    CarmenInletFile = path + '07_CarmenInlet.csv'   

    PRealAge = readWaterAge(PRealFile)
    
    CarmenInletAge = readWaterAge(CarmenInletFile)
   
    CandelariaFile = path + '02_Candelaria.csv'

    ChumpanFile = path + '03_Chumpan.csv'

    CentertFile = path + '04_lagoonCenter.csv'

    CarmenIslandFile = path + '05_CarmenIsland.csv'

    PalizadaFile = path + '06_Palizada.csv'   
                                 
    CandelariaAge = readWaterAge(CandelariaFile)
   
    ChumpanAGe = readWaterAge(ChumpanFile)

    CarmenIslandAge = readWaterAge(CarmenIslandFile)

    PalizadaAge = readWaterAge(PalizadaFile)

    CenterAge = readWaterAge(CentertFile)

    plotSmootWaterAge(CandelariaAge, myWindow, "Candelaria")

    plotSmootWaterAge(ChumpanAGe, myWindow, "Chumpan")

    plotSmootWaterAge(CenterAge, myWindow, "Lagoon Center")

    plotSmootWaterAge(CarmenIslandAge, myWindow, "Carmen Island")

    plotSmootWaterAge(PalizadaAge, myWindow, "Palizada")

    plotSmootWaterAge(CarmenInletAge , myWindow, "Carmen Inlet")

    plotSmootWaterAge(PRealAge, myWindow, "Puerto Real Inlet")
    
    # Save figures 
    plt.savefig('..\\output_files\\age_areas.pdf', dpi=600, format='pdf', metadata={'Author': 'Cardoso-Mohedano JG, 2023'})

    #plt.savefig('single_column_figure.tiff', format='tiff')
   
 
    plt.show()


In [ ]:
def plotTerminosAgesInlets(path, myWindow):
    # Plot the water age time series for the inlets
    files_labels = [
        ('01_PReal_inlet.csv', 'Puerto Real Inlet'),
        ('07_CarmenInlet.csv', 'Carmen Inlet')
    ]
    
    # Loop through each file and label, read the water age, and plot
    for file, label in files_labels:
        water_age = readWaterAge(path + file)
        plotSmootWaterAge(water_age, myWindow, label)
    
    # Save figures

In [ ]:
def plotTerminosAgesZones(path, myWindow):
    # Plot the water age time series for the zones
    files_labels = [
        ('02_Candelaria.csv', 'Candelaria Estuary'),
        ('03_Chumpan.csv', 'Chumpan Estuary'),
        ('04_lagoonCenter.csv', 'Lagoon Center'),
        ('05_CarmenIsland.csv', 'Carmen Island'),
        ('06_Palizada.csv', 'Palizada Estuary')
    ]
    
    # Loop through each file and label, read the water age, and plot
    for file, label in files_labels:
        water_age = readWaterAge(path + file)
        plotSmootWaterAge(water_age, myWindow, label)
    
    # Save figures

In [ ]:

plotTerminosAgesInlets(myPath, window)


In [ ]:
plotTerminosAgesZones(myPath, window)

In [ ]:
import numpy as np

def calculate_mean_std(path):
    # Define file names and corresponding labels
    files_labels = [
        ('01_PReal_inlet.csv', 'Puerto Real Inlet'),
        ('02_Candelaria.csv', 'Candelaria'),
        ('03_Chumpan.csv', 'Chumpan'),
        ('04_lagoonCenter.csv', 'Lagoon Center'),
        ('05_CarmenIsland.csv', 'Carmen Island'),
        ('06_Palizada.csv', 'Palizada'),
        ('07_CarmenInlet.csv', 'Carmen Inlet')
    ]
    
    # Read all files and store in a dictionary
    water_ages = {label: readWaterAge(path + file) for file, label in files_labels}
    
     # Calculate mean and standard deviation for each file
    stats = {label: (float(np.mean(water_age)), float(np.std(water_age))) for label, water_age in water_ages.items()}
    
    # Print mean and standard deviation in a table format
    print(f"{'Location':<20} {'Mean ± Standard Deviation':<30}")
    print("-" * 50)
    for label, (mean, std) in stats.items():
        print(f"{label:<20} {mean:.0f} ± {std:.0f}")
    



    

In [ ]:
calculate_mean_std(myPath)

In [ ]:
import numpy as np

def readWaterAge(file_path):
    # Placeholder function to simulate reading water age data from a CSV file
    # Replace this with actual implementation
    return np.random.rand(100)  # Example: 100 random water age values

def calculate_mean_std(path):
    # Define file names and corresponding labels
    files_labels = [
        ('01_PReal_inlet.csv', 'Puerto Real Inlet'),
        ('02_Candelaria.csv', 'Candelaria'),
        ('03_Chumpan.csv', 'Chumpan'),
        ('04_lagoonCenter.csv', 'Lagoon Center'),
        ('05_CarmenIsland.csv', 'Carmen Island'),
        ('06_Palizada.csv', 'Palizada'),
        ('07_CarmenInlet.csv', 'Carmen Inlet')
    ]
    
    # Read all files and store in a dictionary
    water_ages = {label: readWaterAge(path + file) for file, label in files_labels}
    
    # Calculate mean and standard deviation for each file
    stats = {label: (np.mean(water_age).item(), np.std(water_age, axis=0).item()) for label, water_age in water_ages.items()}
    
    # Print mean and standard deviation in a table format
    print(f"{'Location':<20} {'Mean ± Standard Deviation':<30}")
    print("-" * 50)
    for label, (mean, std) in stats.items():
        print(f"{label:<20} {mean:.2f} ± {std:.2f}")

